In [ ]:
sFile = 'trident.png'
nSize_image = 100 # I recommended a maximum of 150

In [ ]:
# Install necessary packages (if not already installed)
!pip install numpy matplotlib scikit-image scipy networkx pandas pillow

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import matplotlib.collections
from matplotlib import cm
from skimage import io, color, morphology, filters, transform
from scipy.spatial.distance import pdist, squareform
import networkx as nx
import pandas as pd
from IPython.display import display
from google.colab import files

# Step 1: Load and process the image
def load_image(image_path):
    # Read the image
    img = io.imread(image_path)
    # Handle different image shapes
    if len(img.shape) == 3:
        if img.shape[2] == 4:
            # Discard the alpha channel
            img = img[:, :, :3]
        elif img.shape[2] != 3:
            raise ValueError(f"Expected image with 3 channels (RGB) or 4 channels (RGBA), but got {img.shape[2]} channels")
        # Convert to grayscale
        img_gray = color.rgb2gray(img)
    else:
        img_gray = img
    # Resize the image to nSize_imagexnSize_image pixels
    img_gray = transform.resize(img_gray, (nSize_image, nSize_image), anti_aliasing=True)
    # Binarize the image using Otsu's threshold
    thresh = filters.threshold_otsu(img_gray)
    binary_image = img_gray < thresh
    # Remove small artifacts
    binary_image = morphology.remove_small_objects(binary_image, min_size=5)
    return binary_image.astype(int)

# Step 2: Extract black pixel coordinates
def get_black_pixel_coords(binary_image):
    coords = np.column_stack(np.nonzero(binary_image))
    return coords

# Step 3: Generate a path that connects all black pixels
def generate_path_mst(coords):
    if len(coords) == 0:
        raise ValueError("No black pixels found in the image.")
    # Compute the pairwise distance matrix
    dist_matrix = squareform(pdist(coords))
    # Create a fully connected graph
    G = nx.Graph()
    for i in range(len(coords)):
        for j in range(i+1, len(coords)):
            G.add_edge(i, j, weight=dist_matrix[i, j])
    # Compute the Minimum Spanning Tree (MST)
    mst = nx.minimum_spanning_tree(G)
    # Perform a depth-first traversal of the MST to get an approximate path
    path_indices = list(nx.dfs_preorder_nodes(mst, source=0))
    path = coords[path_indices]
    return path

# Step 4: Save the path coordinates to a CSV file
def save_path_to_csv(path, filename='path_coordinates.csv'):
    df = pd.DataFrame(path, columns=['y', 'x'])  # Coordinates are in (row, column) format
    df.to_csv(filename, index=False)
    print(f"Path coordinates saved to {filename}")
    # Optionally, display the first few rows
    display(df.head())

# Step 5: Create an animation of the path and save as a GIF
def create_animation(path):
    num_points = path.shape[0]
    cmap = plt.colormaps['hsv']  # Get the colormap
    colors = cmap(np.linspace(0, 1, num_points))  # Generate colors for each point

    fig, ax = plt.subplots(figsize=(6, 6))

    def update(frame):
        ax.clear()
        ax.set_xlim(0, nSize_image)
        ax.set_ylim(0, nSize_image)
        ax.set_aspect('equal')
        ax.invert_yaxis()  # Invert y-axis to match image coordinates
        ax.axis('off')     # Hide axes
        # Plot the path up to the current frame
        x = path[:frame+1, 1]
        y = path[:frame+1, 0]
        points = np.array([x, y]).T.reshape(-1, 1, 2)
        if len(points) > 1:
            segments = np.concatenate([points[:-1], points[1:]], axis=1)
            lc = matplotlib.collections.LineCollection(segments, colors=colors[:frame+1], linewidth=2)
            ax.add_collection(lc)
        else:
            ax.plot(x, y, marker='o', color='black')  # Single point marker
        return ax,

    ani = animation.FuncAnimation(fig, update, frames=num_points, blit=False, interval=10)

    # Save the animation as a GIF file
    ani.save('tracing.gif', writer='pillow', fps=30)
    print("Animation saved as tracing.gif")

    # Close the figure to prevent display
    plt.close(fig)

    return 'tracing.gif'

# Main function to run the process
def main():
    # Replace 'path/to/your/image.png' with the actual path to your image in Google Drive
    image_path = '/content/drive/MyDrive/' + sFile  # Update this path

    binary_image = load_image(image_path)
    coords = get_black_pixel_coords(binary_image)
    if len(coords) == 0:
        print("No black pixels found in the image. Please check the image or adjust the threshold.")
        return
    # Generate the path using MST and DFS
    path = generate_path_mst(coords)
    # Save the path to a CSV file
    save_path_to_csv(path)
    # Download the CSV file
    try:
        files.download('path_coordinates.csv')
    except Exception as e:
        print(f"Error downloading CSV file: {e}")
    # Create and save the animation as a GIF
    gif_path = create_animation(path)
    # Download the GIF file
    try:
        files.download(gif_path)
    except Exception as e:
        print(f"Error downloading GIF file: {e}")

# Run the main function
main()


Path coordinates saved to path_coordinates.csv


,y,x
0,2,46
1,2,47
2,2,48
3,2,49
4,2,50


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Animation saved as tracing.gif


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>